<a href="https://colab.research.google.com/github/MaralAIHub/ML_Portfolio/blob/main/blob/main/Project_2_SentimentAnalysis/IMDB_Semantic_Analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **تحلیل احساسات با LSTM روی IMDB**

**بارگذاری کتابخانه ها**

In [ ]:
import re
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


**بارگذاری داده**

In [ ]:
dataset = load_dataset("imdb")
print(dataset)

**توکنایزر ساده**

In [ ]:
def tokenize(text):
    return re.findall(r"\w+", text.lower())

# ساخت واژگان (vocab)
counter = Counter()
for txt in dataset["train"]["text"]:
    counter.update(tokenize(txt))

VOCAB_SIZE = 20000   # حداکثر اندازه واژگان
most_common = counter.most_common(VOCAB_SIZE)

vocab = {"<pad>":0, "<unk>":1}
for i, (tok, _) in enumerate(most_common, start=2):
    vocab[tok] = i

pad_idx = vocab["<pad>"]
unk_idx = vocab["<unk>"]
vocab_size = len(vocab)
print("Vocab size:", vocab_size)

# تبدیل متن به لیست عددی
def encode(text):
    return [vocab.get(t, unk_idx) for t in tokenize(text)]

**collate_fn برای DataLoader**

In [ ]:
def collate_batch(batch):
    texts = [torch.tensor(encode(item["text"]), dtype=torch.long) for item in batch]
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)
    texts_padded = nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=pad_idx)
    return labels, texts_padded

train_loader = DataLoader(dataset["train"], batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(dataset["test"],  batch_size=64, shuffle=False, collate_fn=collate_batch)

**تعریف مدل LSTM**

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128, output_dim=2, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        embedded = self.embedding(x)                # (B, L, E)
        _, (hidden, _) = self.lstm(embedded)        # hidden: (num_layers, B, H)
        out = self.fc(self.dropout(hidden[-1]))     # آخرین لایه LSTM
        return out

model = LSTMClassifier(vocab_size=vocab_size, pad_idx=pad_idx).to(device)
print(model)

**آموزش**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 2   # برای تست سریع، بیشتر کنید برای دقت بالاتر
history = {"loss":[], "acc":[]}

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    correct = total = 0
    for labels, texts in train_loader:
        labels = labels.to(device)
        texts = texts.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    avg_loss = total_loss/len(train_loader)
    acc = correct/total
    history["loss"].append(avg_loss)
    history["acc"].append(acc)
    print(f"Epoch {epoch+1}  Loss={avg_loss:.3f}  Acc={acc:.3f}")

**نمودار آموزش**

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(range(1, epochs+1), history["loss"], marker='o')
plt.title("Loss")
plt.subplot(1,2,2)
plt.plot(range(1, epochs+1), history["acc"], marker='o')
plt.title("Accuracy")
plt.show()

**پیش‌بینی روی جمله جدید**

In [ ]:
def predict_sentiment(text):
    model.eval()
    with torch.no_grad():
        ids = torch.tensor(encode(text), dtype=torch.long).unsqueeze(0).to(device)
        out = model(ids)
        probs = F.softmax(out, dim=1).cpu().numpy()[0]
        return probs

examples = ["This movie was amazing!", "I hated this movie, it was so boring."]
for s in examples:
    p = predict_sentiment(s)
    print(s, "-> neg:{:.2f}, pos:{:.2f}".format(p[0], p[1]))
